In [1]:
import tensorflow as tf
import pprint
import numpy as np

# Log in to your W&B account
!pip install wandb
import wandb
wandb.login()
# API: 18fe909a998e642a33986bd03c61ee2c6922f72d


# Loading the MNIST dataset in one line
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train_normalized = x_train/255
x_test_normalized = x_test/255
x_train= x_train_normalized.reshape(-1, 28, 28, 1)
x_test = x_test_normalized.reshape(-1, 28, 28, 1)

# Printing the shape
print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


11490434/11490434 [==============================] - 0s 0us/step
x_train: (60000, 28, 28, 1)
y_train: (60000,)
x_test: (10000, 28, 28, 1)
y_test: (10000,)


In [ ]:
def Net(neurons1=26, neurons2=26, neurons3=54, acivation1='relu',activation2='relu',activation3='sigmoid',do1=0.2, do2=0.2, do3=0.2, do4=0.2, gn1=0.1):
    return tf.keras.models.Sequential(
    [
    tf.keras.layers.InputLayer(input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(filters=neurons1, kernel_size=5, padding='same', activation=acivation1),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(do1),
    tf.keras.layers.Conv2D(filters=neurons2, kernel_size=5, padding='same', activation=activation2, kernel_initializer=tf.keras.initializers.HeNormal()), #new
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(do2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(do3),
    tf.keras.layers.Dense(neurons3, kernel_regularizer = tf.keras.regularizers.l2(0.07), activation = activation3),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(do4),
    tf.keras.layers.GaussianNoise(gn1),
    tf.keras.layers.Dense(10, activation='softmax')
    ]
)

model = Net()
model.summary()

def get_optimizer(lr=1e-3, optimizer="adam"):
    "Select optmizer between adam and sgd with momentum"
    if optimizer.lower() == "adam":
        return tf.keras.optimizers.Adam(learning_rate=lr)
    if optimizer.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.1)

def train(model, batch_size=64, epochs=10, lr=1e-3, optimizer='adam', log_freq='epoch'):

    # Compile model like you usually do.
    tf.keras.backend.clear_session()

    # Compile modell
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy() #from_logits=True
    sampleID = 100
    loss_fn(y_train[:1], model(x_train[sampleID-1:sampleID]).numpy()).numpy()

    model.compile(loss=loss_fn,
                  optimizer=get_optimizer(lr, optimizer),
                  metrics=['accuracy'])

    # callback setup
    wandb_callbacks = [wandb.keras.WandbCallback(log_freq=log_freq,save_model=(False),) ]

    # Train your model
    model.fit(
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data = (x_test, y_test),
        callbacks = [wandb_callbacks]
        )

# Configure Sweep
sweep_config = {
    'method': 'bayes',
    'metric': {
        'goal': 'maximize',
        'name': 'val_accuracy'
    },
    'early_terminate': {
        "type": "hyperband",
        "eta": 2,
        "min_iter":3
     }
}
parameters_dict = {
    'optimizer': {
        'value': 'adam'
        },
    'epochs': {
        'value': 20
        },
    'batch_size' : {
        'value': 32
        },
    'learning_rate': {
        'value': 1e-5
    },
     'neurons1': {
        'value': 26
    },
    'neurons2': {
        'value': 26
    },
    'neurons3': {
        'value': 54
    },

    'activation1' : {
        'value' :  ['relu']
    },
    'activation2' : {
        'value' : ['relu']
    },
    'activation3' : {
        'value' : ['sigmoid']
    },
    'do1' : {
        'values' : list(np.arange(0.01,0.8,0.05))
    },
    'do2' : {
        'values' : list(np.arange(0.01,0.8,0.05))
    },
    'do3' : {
        'values' : list(np.arange(0.01,0.8,0.05))
    },
    'do4' : {
        'values' : list(np.arange(0.01,0.8,0.05))
    },
    'gn1' : {
        'values' : list(np.arange(0.01,0.8,0.05))
    }
}
sweep_config['parameters'] = parameters_dict
pprint.pprint(sweep_config)

def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project name
    with wandb.init(config=config_defaults):  # this gets over-written in the Sweep

        # Specify the other hyperparameters to the configuration, if any
        wandb.config.architecture_name = "V03"
        wandb.config.dataset_name = "MNIST"

        # initialize model
        model = Net(wandb.config.neurons1, wandb.config.neurons2, wandb.config.neurons3)

        train(model,
              wandb.config.batch_size,
              wandb.config.epochs,
              wandb.config.learning_rate,
              wandb.config.optimizer)

sweep_id = wandb.sweep(sweep_config, project="V03_Dropout")
wandb.agent(sweep_id, function=sweep_train, count=30)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 26)        676       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 26)       104       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 26)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 26)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 26)        16926     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 26)         0

wandb: Agent Starting Run: 04idbua1 with config:
wandb: 	activation1: ['relu']
wandb: 	activation2: ['relu']
wandb: 	activation3: ['sigmoid']
wandb: 	batch_size: 32
wandb: 	do1: 0.41
wandb: 	do2: 0.6100000000000001
wandb: 	do3: 0.7100000000000001
wandb: 	do4: 0.7100000000000001
wandb: 	epochs: 20
wandb: 	gn1: 0.16000000000000003
wandb: 	learning_rate: 1e-05
wandb: 	neurons1: 26
wandb: 	neurons2: 26
wandb: 	neurons3: 54
wandb: 	optimizer: adam
wandb: Currently logged in as: lucas-halt (luchalt). Use `wandb login --relogin` to force relogin


Epoch 1/20


2023-11-06 15:22:31.617065: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1875/1875 [==============================] - 13s 5ms/step - loss: 8.3460 - accuracy: 0.3794 - val_loss: 6.3485 - val_accuracy: 0.8241
Epoch 2/20
1875/1875 [==============================] - 8s 4ms/step - loss: 5.9181 - accuracy: 0.6872 - val_loss: 4.6512 - val_accuracy: 0.9038
Epoch 3/20
1875/1875 [==============================] - 8s 4ms/step - loss: 4.2983 - accuracy: 0.8033 - val_loss: 3.3632 - val_accuracy: 0.9337
Epoch 4/20
1875/1875 [==============================] - 8s 4ms/step - loss: 3.0896 - accuracy: 0.8634 - val_loss: 2.3988 - val_accuracy: 0.9492
Epoch 5/20
1875/1875 [==============================] - 8s 4ms/step - loss: 2.2188 - accuracy: 0.8934 - val_loss: 1.7142 - val_accuracy: 0.9553
Epoch 6/20
1875/1875 [==============================] - 8s 4ms/step - loss: 1.6105 - accuracy: 0.9141 - val_loss: 1.2423 - val_accuracy: 0.9631
Epoch 7/20
1875/1875 [==============================] - 8s 4ms/step - loss: 1.1962 - accuracy: 0.9258 - val_loss: 0.9188 - val_accuracy: 0.9663
Ep

...

accuracy,▁▅▆▇▇▇▇█████████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▆▅▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▅▆▆▇▇▇▇▇██████████
val_loss,█▆▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.97198
best_epoch,19
best_val_loss,0.10094
epoch,19
loss,0.15763
val_accuracy,0.9864


wandb: Agent Starting Run: u3gb25yl with config:
wandb: 	activation1: ['relu']
wandb: 	activation2: ['relu']
wandb: 	activation3: ['sigmoid']
wandb: 	batch_size: 32
wandb: 	do1: 0.7100000000000001
wandb: 	do2: 0.6100000000000001
wandb: 	do3: 0.060000000000000005
wandb: 	do4: 0.56
wandb: 	epochs: 20
wandb: 	gn1: 0.36000000000000004
wandb: 	learning_rate: 1e-05
wandb: 	neurons1: 26
wandb: 	neurons2: 26
wandb: 	neurons3: 54
wandb: 	optimizer: adam


Epoch 1/20


2023-11-06 16:56:40.911801: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1875/1875 [==============================] - 11s 4ms/step - loss: 8.4056 - accuracy: 0.3620 - val_loss: 6.3609 - val_accuracy: 0.8036
Epoch 2/20
1875/1875 [==============================] - 8s 4ms/step - loss: 5.9480 - accuracy: 0.6610 - val_loss: 4.6408 - val_accuracy: 0.8893
Epoch 3/20
1875/1875 [==============================] - 8s 4ms/step - loss: 4.3116 - accuracy: 0.7817 - val_loss: 3.3480 - val_accuracy: 0.9237
Epoch 4/20
1875/1875 [==============================] - 8s 4ms/step - loss: 3.1077 - accuracy: 0.8437 - val_loss: 2.3934 - val_accuracy: 0.9426
Epoch 5/20
1875/1875 [==============================] - 8s 4ms/step - loss: 2.2358 - accuracy: 0.8814 - val_loss: 1.7104 - val_accuracy: 0.9527
Epoch 6/20
1875/1875 [==============================] - 8s 4ms/step - loss: 1.6196 - accuracy: 0.9068 - val_loss: 1.2344 - val_accuracy: 0.9608
Epoch 7/20
1875/1875 [==============================] - 8s 4ms/step - loss: 1.1982 - accuracy: 0.9222 - val_loss: 0.9075 - val_accuracy: 0.9662
Ep

In [ ]:
# after the training finishes, we will also save Marvin in Keras style (HDF5), so we do not have to
# train him again
# every time we start our computer. Obviously, by changing the model_name, you can also save different
# configurations of Marvin. The name has to be a string, like this: 'name.h5'
model_name = 'pepsi_p62k_a9915'
model.save(model_name, save_format='h5')

# It is best practice to indicate what configuration changes you did within the name, so you know
# which model you need to load already from its name
# Let's say instead of a learning rate of 0.001 you used 0.1, your naming could then look like:
# 'marvin_lr01.h5'

print('Success! You saved Marvin as: ', model_name)

In [ ]:
# load a saved marvin configuration you want to evaluate
model_name = 'pepsi_lr03'
pepsi_reloaded = tf.keras.models.load_model(model_name)

# Let Marvin predict on the test set, so we have some data to evaluate his performance.
predictions = pepsi_reloaded.predict([x_test])
# predictions = pepsi.predict([x_test])

# Remember that the prediction of Marvin is a probability distribution over all ten-digit classes
# We want him to assign the digit class with the highest probability to the sample.
predictions = np.argmax(predictions, axis=1)
#pd.DataFrame(predictions)

# Plot for the intuitive approach

numbers_to_display = 196
num_cells = math.ceil(math.sqrt(numbers_to_display))
plt.figure(figsize=(15, 15))

for plot_index in range(numbers_to_display):
    predicted_label = predictions[plot_index]
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    color_map = 'Greens' if predicted_label == y_test[plot_index] else 'Reds'
    plt.subplot(num_cells, num_cells, plot_index + 1)
    plt.imshow(x_test_normalized[plot_index].reshape((28, 28)), cmap=color_map)
    plt.xlabel(predicted_label)

plt.subplots_adjust(hspace=1, wspace=0.5)
plt.show()

In [ ]:
confusion_matrix = tf.math.confusion_matrix(y_test, predictions)

f, ax = plt.subplots(figsize=(9, 7))
sn.heatmap(
    confusion_matrix,
    annot=True,
    linewidths=.7,
    fmt="d",
    square=True,
    ax=ax,
    cmap="viridis",
)
plt.show()